# 这个是处理GPT2-nano可视化注意力的数据集
### 核心就是大于300的句子保留，其他截断，不合并短句子，只有验证集

In [1]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer

# Load WikiText-103 dataset
# dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")

dataset = load_from_disk("../wikitext-103-raw-train")

In [3]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2") # change your own model
print(tokenizer.vocab_size)
print("Tokenizer max length: ", tokenizer.model_max_length)  # change your own model max input length
tokenizer.model_max_length = 512
print("Tokenizer max length after change: ", tokenizer.model_max_length) 

# Some tokenizer have a pad_token, which is used to pad a sequence up to max_length. But GPT2 tokenizer doesn't have it
if tokenizer.pad_token is None:
    # tokenizer.set_special_tokens({'pad_token': '[PAD]'})
    tokenizer.pad_token = tokenizer.eos_token

# convenient function to load tokenizer next time
tokenizer.save_pretrained(f"../tokenizer_save/tokenizer-{tokenizer.name_or_path}-{tokenizer.model_max_length}")

Using pad_token, but it is not set yet.


50257
Tokenizer max length:  1024
Tokenizer max length after change:  512


('../tokenizer_save/tokenizer-gpt2-512/tokenizer_config.json',
 '../tokenizer_save/tokenizer-gpt2-512/special_tokens_map.json',
 '../tokenizer_save/tokenizer-gpt2-512/vocab.json',
 '../tokenizer_save/tokenizer-gpt2-512/merges.txt',
 '../tokenizer_save/tokenizer-gpt2-512/added_tokens.json')

In [4]:
# make your own filter function
def filter_function(example):  
    return len(example['text'].split()) >= 300

dataset_without_short = dataset.filter(filter_function,num_proc=4)

Filter (num_proc=4):   0%|          | 0/1801350 [00:00<?, ? examples/s]

In [5]:
print(len(dataset_without_short))

19502


In [6]:
# Define function to tokenize and encode text
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=tokenizer.model_max_length)
    

# Apply preprocessing to dataset
preprocessed_dataset_without_short = dataset_without_short.map(preprocess_function, batched=True, num_proc=4)



Map (num_proc=4):   0%|          | 0/19502 [00:00<?, ? examples/s]

In [7]:
# Split preprocessed dataset into train, validation, and test sets
# splits = preprocessed_dataset_without_short.train_test_split(test_size=0.1, shuffle=True)
preprocessed_splits = DatasetDict({
    "validation": preprocessed_dataset_without_short,
    "test": load_dataset("wikitext", "wikitext-103-raw-v1", split="test")
})

Found cached dataset wikitext (/home/songx_lab/cse12012530/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [9]:
preprocessed_splits.save_to_disk("../wikitext-103-prepro-gpt2-512-wo-train-for-visualize")

Saving the dataset (0/1 shards):   0%|          | 0/19502 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4358 [00:00<?, ? examples/s]